In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import dash
from dash import html,dcc
from dash.dependencies import Input, Output, State
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
                               
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),  
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                             placeholder="Select a Launch Site",
                                             searchable=True),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div([ ],id='success-pie-chart'),
                                html.Br(),

                                html.P("Payload range (Kg):"),
#                                 # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, 
                                                max=10000, 
                                                step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '1E4'},
                                                value=[min_payload, max_payload],
                                                tooltip={'always_visible':True}),

#                                 TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div([ ],id='payload-chart'),
                                ])



In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='children'),
              Input(component_id='site-dropdown', component_property='value'),
              State(component_id='success-pie-chart', component_property='children'))

def get_pie_chart(entered_site,c1):
    
    if entered_site == 'ALL':
        data = spacex_df
        titulo='by launch site'
        fig = px.pie(data, 
                     values='class',
                     names='Launch Site',
                     title='Total success rate '+titulo)
    else:
        data = spacex_df[spacex_df['Launch Site']==entered_site].groupby('class').mean()
        
        titulo='Total success rate for site '+entered_site
        
        rates=[spacex_df.groupby('Launch Site')['class'].mean().loc[entered_site],
               1-spacex_df.groupby('Launch Site')['class'].mean().loc[entered_site]]
        
        labels=[1,0]
        df_pie=pd.DataFrame({'etiquetas':labels,'valores':rates})
        
        fig = px.pie(df_pie, 
                     values='valores',
                     names='etiquetas',
                     title=titulo)
    
    return dcc.Graph(figure=fig)
        
@app.callback(Output(component_id='payload-chart', component_property='children'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'),
              State(component_id='payload-chart', component_property='children'))

def get_scatter_chart(entered_site,slider,c1):
    
    val_min=slider[0]
    val_max=slider[1]
    scatter_data=spacex_df[spacex_df['Payload Mass (kg)']>=val_min]
    scatter_data=scatter_data[scatter_data['Payload Mass (kg)']<=val_max]
    
    if entered_site == 'ALL':
        
        fig_scatter=px.scatter(data_frame=scatter_data,
                               x='Payload Mass (kg)',
                               y='class',
                               color="Booster Version Category",
                               title='Correlation between payload (Kg) and Success landing for all sites')
    else:
        
        fig_scatter=px.scatter(data_frame=scatter_data[scatter_data['Launch Site']==entered_site],
                               x='Payload Mass (kg)',
                               y='class',
                               color="Booster Version Category",
                               title='Correlation between payload (Kg) and Success landing for site: '+entered_site)
    
    return dcc.Graph(figure=fig_scatter)

In [6]:
# TASK 3:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [18/Jan/2023 11:54:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2023 11:54:43] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\dash\dash.py", line 1280, in dispatch
    callback_context=g,
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\dash\_callba

127.0.0.1 - - [18/Jan/2023 11:54:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\dash\dash.py", line 1280, in dispatch
    callback_context=g,
  File "C:\Users\AN012491\AppData\Local\Continuum\anaconda3\lib\site-packages\dash\_callba

127.0.0.1 - - [18/Jan/2023 11:54:43] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Jan/2023 11:54:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:48] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2023 11:54:48] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2023 11:54:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2023 11:54:54] "POST /_dash-update-component HTTP/1.1" 200 -
